<a href="https://colab.research.google.com/github/FIIT-Bc-projects/2022-Petrik-AbstractiveSummarization/blob/main/research/cnn_prepr_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN daily mail processing testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## CNN dataset load